In [85]:
from src.util import connect_to_db
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# level_modifiers = [x/105 for x in [0, 105, 250, 595, 1414, 3360, 8000, 19000, 45255, 107700, 256000]]
level_modifiers = {
    0: 0,
    1: 1/105,
    2: 250/105,
    3: 595/105,
    4: 1414/105,
    5: 3360/105,
    6: 8000/105,
    7: 19000/105,
    8: 45255/105,
    9: 107700/105,
    10: 256000/105    
}

def records_to_df(records):
    return pd.DataFrame([dict(r) for r in records])

conn = await connect_to_db()

contracts = records_to_df(await conn.fetch("SELECT * FROM esi.contracts"))
contract_items = records_to_df(await conn.fetch("SELECT * FROM esi.contract_items"))

type_ids = records_to_df(await conn.fetch("SELECT * FROM sde.type_ids"))

market_aggregates = records_to_df(await conn.fetch("SELECT * FROM market.aggregates WHERE location_id = 60003760"))
blueprints = records_to_df(await conn.fetch("SELECT * FROM sde.blueprints"))

contracts["count"] = contracts["contract_id"].map(contract_items.groupby("contract_id").size())
contracts = contracts[contracts["count"] == 1]

contracts = contracts.merge(contract_items, on="contract_id")

contracts = contracts[contracts["type"] == 'item_exchange']
contracts = contracts[contracts["is_included"]]
contracts = contracts[contracts["material_efficiency"].notna()]
contracts = contracts[~contracts["is_blueprint_copy"]]

contracts = contracts.merge(type_ids, left_on="type_id", right_on="type_id")
contracts = contracts.merge(blueprints, left_on="type_id", right_on="blueprint_type_id")

contracts["premium"] = contracts["price"] - contracts["base_price"]
contracts["research_time"] = contracts["time_efficiency"].map(lambda x: level_modifiers[int(x/2)]) * contracts["research_time_time"] \
    + contracts["material_efficiency"].map(lambda x: level_modifiers[int(x)]) * contracts["research_material_time"]

contracts["research_time_premium"] = contracts["premium"].map(float) / contracts["research_time"]
contracts = contracts[np.logical_and(contracts["research_time_premium"].notna(), contracts["research_time_premium"] != float("-inf"))]

contracts[["contract_id", "type_id", "name", "base_price", "price", "premium", "is_blueprint_copy", "material_efficiency", "time_efficiency", "research_time", "research_time_premium"]].sort_values("research_time_premium", ascending=False).head(30)


# contracts.plot(kind="scatter", x="research_time_premium", y="price", alpha=0.1, figsize=(10, 10), logx=True, logy=True)



,contract_id,type_id,name,base_price,price,premium,is_blueprint_copy,material_efficiency,time_efficiency,research_time,research_time_premium
3477,199415358,11621,Tracking Computer I Blueprint,99000.00,15000000.00,14901000.00,False,0.0,0.0,0.0,inf
819,200201445,26027,Large Bay Loading Accelerator I Blueprint,1250000.00,1200000000.00,1198750000.00,False,0.0,0.0,0.0,inf
3480,199415370,11621,Tracking Computer I Blueprint,99000.00,15000000.00,14901000.00,False,0.0,0.0,0.0,inf
4155,200037469,46359,Rare Moon Mining Crystal Type A I Blueprint,7617600.00,23000000.00,15382400.00,False,0.0,0.0,0.0,inf
3479,199415365,11621,Tracking Computer I Blueprint,99000.00,15000000.00,14901000.00,False,0.0,0.0,0.0,inf
3478,199415363,11621,Tracking Computer I Blueprint,99000.00,15000000.00,14901000.00,False,0.0,0.0,0.0,inf
2362,200114518,941,Incursus Blueprint,2825000.00,25000000.00,22175000.00,False,0.0,0.0,0.0,inf
565,200152482,24703,Hurricane Blueprint,565000000.00,650000000.00,85000000.00,False,0.0,0.0,0.0,inf
1566,199708600,31443,Small Energy Burst Aerator I Blueprint,125000.00,34900000.00,34775000.00,False,0.0,0.0,0.0,inf
499,200046513,25951,Large Egress Port Maximizer I Blueprint,1250000.00,25000000.00,23750000.00,False,0.0,0.0,0.0,inf


e:\Github\eve-api-db\venv\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
